# APIM ❤️ 3rd Party models

## Gemini Models lab
![flow](../../images/gemini-models.gif)

Playground to try Gemini Models served trough the AI Gateway. This lab creates two API's: one with native Gemini API compatibility and another one with OpenAI compatibility. 

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)
- [Gemini API Key from Google](https://aistudio.google.com/apikey)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`... 


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the models and versions according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}, 
                             {"name": "subscription2", "displayName": "Subscription 2"}, 
                             {"name": "subscription3", "displayName": "Subscription 3"}]

openai_compatible_api_path = "openaicompatible"  # path to the inference API in the APIM service with OpenAI compatibility
gemini_inference_api_path = "geminiapi"  # path to the inference API in the APIM service with Google GenAI compatibility
gemini_api_url = "https://generativelanguage.googleapis.com"  # replace with your Gemini API URL
gemini_api_version = "v1beta"
gemini_model_name = "gemini-3-flash-preview"  # replace with your desired Gemini model name
gemini_api_key = "your-gemini-api-key"  # replace with your Gemini API key

utils.print_ok('Notebook initialized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [ ]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "openAICompatibleAPIPath": { "value": openai_compatible_api_path },
        "geminiInferenceAPIPath": { "value": gemini_inference_api_path },
        "geminiAPIURL": { "value": gemini_api_url },
        "geminiAPIKey": { "value": gemini_api_key }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key

<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior and troubleshoot the [policy](policy.xml).

In [ ]:
import json, requests, time
url = f"{apim_resource_gateway_url}/{gemini_inference_api_path}/{gemini_api_version}/models/{gemini_model_name}:generateContent"

contents = {
    "contents": [
      {
        "parts": [
          {
            "text": "Explain how AI works in a few words"
          }
        ]
      }
    ]
  }

# Initialize a session for connection pooling and set any default headers
session = requests.Session()
session.headers.update({
    'x-goog-api-key': api_key,
    'x-user-id': 'alex'
})

try:
    start_time = time.time()
    response = session.post(url, json = contents)
    response_time = time.time() - start_time
    print(f"⌚ {response_time:.2f} seconds")

    utils.print_response_code(response)
    print(f"Response headers: {json.dumps(dict(response.headers), indent = 4)}")

    print(f"{response.text}\n")

finally:
    # Close the session to release the connection
    session.close()


<a id='googlesdk'></a>
### 🧪 Test the inference API with the Google GenAI SDK


In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=api_key,
    http_options=types.HttpOptions(
        base_url=f"{apim_resource_gateway_url}/{gemini_inference_api_path}",
        api_version=gemini_api_version,
    )
)

response = client.models.generate_content(
    model=gemini_model_name, contents="Explain how AI works in a few words"
)
print(response.text)

<a id='sdk'></a>
### 🧪 Test with streaming using the OpenAI Python SDK

Notice that the base_url is appended with `openai` to ensure OpenAI API compatibility.   
With a streaming API call, the response is sent back incrementally in chunks via an [event stream](https://developer.mozilla.org/docs/Web/API/Server-sent_events/Using_server-sent_events#event_stream_format). In Python, you can iterate over these events with a for loop.

In [ ]:
import time
from openai import OpenAI
client = OpenAI(
    api_key=api_key, # the api key will be sent as an authorization bearer token. You need to set the 'bearer' enabled in the APIM api settings.
    base_url=f"{apim_resource_gateway_url}/{openai_compatible_api_path}/{gemini_api_version}/openai"
)
response = client.chat.completions.with_raw_response.create(
    model=gemini_model_name,
    messages=[{"role": "user", "content": "Count to 100, with a comma between each number and no newlines. E.g., 1, 2, 3, ..."}],
    stream=True
)

print("headers ", response.headers)

completion = response.parse() 

# create variables to collect the stream of chunks
collected_chunks = []
collected_messages = []
# iterate through the stream of events
for chunk in completion:
    chunk_time = time.time() - start_time  # calculate the time delay of the chunk
    collected_chunks.append(chunk)  # save the event response
    if chunk.choices:
        chunk_message = chunk.choices[0].delta.content  # extract the message
        collected_messages.append(chunk_message)  # save the message
        print(f"Message received {chunk_time:.2f} seconds after request: {chunk_message}")  # print the delay and text
# print the time delay and text received
print(f"Full response received {chunk_time:.2f} seconds after request")  # type: ignore
# clean None in collected_messages
collected_messages = [m for m in collected_messages if m is not None]
full_reply_content = ''.join(collected_messages)
print(f"Full conversation received: {full_reply_content}")




<a id='sdk'></a>
### 🧪 Execute multiple runs for each subscription using the OpenAI Python SDK

We will send requests for each subscription. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario.


In [ ]:
import time
from openai import OpenAI

runs = 10
sleep_time_ms = 100
base_url=f"{apim_resource_gateway_url}/{openai_compatible_api_path}/{gemini_api_version}/openai"

clients = [
    OpenAI(api_key=apim_subscriptions[0].get("key"), base_url=base_url),
    OpenAI(api_key=apim_subscriptions[1].get("key"), base_url=base_url),
    OpenAI(api_key=apim_subscriptions[2].get("key"), base_url=base_url)
]

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    for j in range(0, 3):
        response = clients[j].chat.completions.create(
                model=gemini_model_name,
                messages = [
                    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
                    {"role": "user", "content": "Can you tell me the time, please?"}
                ],
                extra_headers = {"x-user-id": "alex"}
            )

        print(f"💬 Subscription {j+1}: {response.choices[0].message.content}")

    print()

    time.sleep(sleep_time_ms/1000)


<a id='kql'></a>
### 🔍 Display LLM logging


In [ ]:
import pandas as pd

query = "let llmHeaderLogs = ApiManagementGatewayLlmLog \
        | where DeploymentName != ''; \
        let llmLogsWithSubscriptionId = llmHeaderLogs \
        | join kind=leftouter ApiManagementGatewayLogs on CorrelationId \
        | project \
            SubscriptionId = ApimSubscriptionId, DeploymentName, ModelName, TotalTokens; \
        llmLogsWithSubscriptionId \
        | summarize \
            SumTotalTokens      = sum(TotalTokens) \
        by SubscriptionId, DeploymentName, ModelName"

output = utils.run(f"az monitor log-analytics query -w {log_analytics_id} --analytics-query \"{query}\"", "Retrieved log analytics query output", "Failed to retrieve log analytics query output") 
if output.success and output.json_data:
    table = output.json_data
    display(pd.DataFrame(table))


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.